In [1]:
import numpy as np
import pandas as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
import os
import random
import glob
from shutil import copyfile
from sklearn.model_selection import train_test_split

ModuleNotFoundError: No module named 'pandas'

In [ ]:
main_path='../input/microsoft-catsvsdogs-dataset/PetImages'
folders=[]
for folder in os.listdir(main_path):
    folders.append(folder)
print(folders)


In [4]:
os.mkdir('./data')
os.mkdir('./data/train')
os.mkdir('./data/valid')
for folder in folders:
    files=os.listdir(os.path.join(main_path,folder))
    images=[]
    for img in files:
        try:
            img_arr=tf.io.read_file(os.path.join(main_path,folder,img))
            img_arr=tf.image.decode_image(img_arr,channels=3)
            if img_arr.ndim==3:
                images.append(img)
        except:
            pass
    random.shuffle(images)
    count=len(images)
    split=int(0.8*count)
    os.mkdir(os.path.join('./data/train',folder))
    os.mkdir(os.path.join('./data/valid',folder))
    for c in range(split):
        source_file=os.path.join(main_path,folder,images[c])
        destination=os.path.join('./data/train',folder,images[c])
        copyfile(source_file,destination)
    for c in range(split,count):
        source_file=os.path.join(main_path,folder,images[c])
        destination=os.path.join('./data/valid',folder,images[c])
        copyfile(source_file,destination)

In [8]:
train_path='./data/train'
valid_path='./data/valid'

In [11]:
train_batches=keras.preprocessing.image.ImageDataGenerator().flow_from_directory(
    train_path,
    target_size=(224,224),
    batch_size=10
)

valid_batches=keras.preprocessing.image.ImageDataGenerator().flow_from_directory(
    valid_path,
    target_size=(224,224),
    batch_size=30
)

In [ ]:
from keras.applications.vgg16 import VGG16
base_model = VGG16(weights='imagenet' , include_top = False , 
                  input_shape=(224,224,3))

In [ ]:
base_model.summary()

In [ ]:
for layer in base_model.layers:
  layer.trainable=False

In [ ]:
last_layer = base_model.get_layer('block5_pool')
last_out=last_layer.output
x=keras.layers.Flatten()(last_out)
x=keras.layers.Dense(64,activation='relu')(x)
x=keras.layers.BatchNormalization()(x)
x=keras.layers.Dropout(0.25)(x)
x=keras.layers.Dense(2,activation='sigmoid')(x)

In [ ]:
model=keras.Model(inputs=base_model.input , outputs=x)

In [ ]:
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [ ]:
model.fit(train_batches,validation_data=valid_batches,epochs=20)